<a href="https://colab.research.google.com/github/myungseonSim/kakao-tech/blob/main/GenAi/KTB_Day19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install transformers
!pip install sentence-transformers
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu

In [ ]:
import torch
from transformers import DPRQuestionEncoderTokenizer, DPRQuestionEncoder, DPRContextEncoderTokenizer, DPRContextEncoder
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
# 간단한 문서 데이터 준비
documents = [
    "Paris is the capital of France.",
    "Berlin is the capital of Germany.",
    "Madrid is the capital of Spain.",
    "Rome is the capital of Italy.",
    "London is the capital of the United Kingdom."
    "goorm is the capital of the AI Kingdom"
]

In [ ]:
# 문서 인코더와 토크나이저 준비
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# 문서를 임베딩으로 변환
document_embeddings = []
for doc in documents:
    inputs = context_tokenizer(doc, return_tensors="pt")
    outputs = context_encoder(**inputs)
    document_embeddings.append(outputs.pooler_output.detach().numpy())

In [ ]:
# 리스트 형태의 임베딩을 하나의 numpy 배열로 변환
document_embeddings = np.vstack(document_embeddings)

In [ ]:
print(document_embeddings[0])

[ 2.29594056e-02 -3.66184354e-01 -1.31380007e-01 -6.76845163e-02
 -1.00397415e-01  2.83430994e-01  2.66060859e-01  9.53448594e-01
 -6.34384930e-01 -7.05658674e-01 -5.86966872e-01 -2.38292944e-03
 -3.80576551e-02  3.89092058e-01  2.33061120e-01  1.10080570e-01
  5.80546200e-01  9.76435542e-02  6.11330792e-02 -2.23775744e-01
 -5.44697165e-01  4.60170060e-01  1.89224660e-01 -2.73531377e-01
  3.40862006e-01 -3.40622962e-01 -5.32925725e-01  8.51325020e-02
  2.53665030e-01  3.36141109e-01 -5.36819100e-02 -1.06614895e-01
 -4.89022791e-01  6.81112036e-02  1.69389606e-01 -4.13111523e-02
  2.87471116e-01 -8.42847005e-02  2.00074822e-01 -4.30326551e-01
  3.99038196e-02 -2.93828756e-01  2.56182581e-01  6.43227473e-02
 -2.48362124e-01 -4.82591331e-01 -6.55608654e-01  6.36978328e-01
 -2.96619892e-01 -2.26479080e-02 -3.53044331e-01 -1.84896179e-02
  5.77839017e-02  8.35039496e-01  1.91975743e-01  1.92332208e-01
 -9.60320294e-01  2.83110648e-01 -2.36994818e-01 -1.82171345e-01
  9.83215123e-02  5.69320

https://github.com/facebookresearch/faiss


FAISS (Facebook AI Similarity Search) is a library that allows developers to quickly search for embeddings of multimedia documents that are similar to each other. It solves limitations of traditional query search engines that are optimized for hash-based searches, and provides more scalable similarity search functions.

In [ ]:
# FAISS 인덱스 생성 및 임베딩 추가
index = faiss.IndexFlatL2(document_embeddings.shape[1])
index.add(document_embeddings)

In [ ]:
# 질문 인코더와 토크나이저 설정
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

# 생성 모델과 토크나이저 설정
generation_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
generation_model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
print(generation_model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)


In [ ]:
# RAG 모델 클래스 정의
class RAGModel:
    def __init__(self, question_encoder, context_encoder, generation_model, index, tokenizer):
        self.question_encoder = question_encoder
        self.context_encoder = context_encoder
        self.generation_model = generation_model
        self.index = index
        self.tokenizer = tokenizer

    def retrieve(self, question):
        # 질문을 인코딩하여 임베딩으로 변환
        question_inputs = question_tokenizer(question, return_tensors="pt")
        question_outputs = self.question_encoder(**question_inputs)
        question_embedding = question_outputs.pooler_output.detach().numpy()

        # FAISS 인덱스를 사용하여 가장 가까운 문서 검색
        D, I = self.index.search(question_embedding, k=1)  # 가장 가까운 문서 검색
        return documents[I[0][0]]

    def generate(self, question, retrieved_doc):
        # 질문과 검색된 문서를 결합하여 입력으로 사용
        input_text = question + " " + retrieved_doc
        inputs = self.tokenizer.encode(input_text, return_tensors="pt")

        # 모델을 통해 결과 출력
        outputs = self.generation_model.generate(inputs, max_length=50)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def run(self, question):
        # 검색된 문서를 기반으로 답변 생성
        retrieved_doc = self.retrieve(question)
        print(f"Retrieved Document: {retrieved_doc}")
        return self.generate(question, retrieved_doc)

# RAG 모델 인스턴스 생성
rag_model = RAGModel(question_encoder, context_encoder, generation_model, index, generation_tokenizer)

In [ ]:
# 테스트 함수
def test_rag_model(question):
    answer = rag_model.run(question)
    print(f"Question: {question}")
    print(f"Answer: {answer}")

def generate_pure_answer(question):
    inputs = generation_tokenizer.encode(question, return_tensors="pt")
    outputs = generation_model.generate(inputs, max_length=50)
    return generation_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# 예제 질문
question = "What is the capital of the AI Kingdom?"
test_rag_model(question)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Retrieved Document: London is the capital of the United Kingdom.goorm is the capital of the AI Kingdom
Question: What is the capital of the AI Kingdom?
Answer: What is the capital of the AI Kingdom? London is the capital of the United Kingdom.goorm is the capital of the AI Kingdom.

Goorm is the capital of the AI Kingdom.

Goorm is the capital of the AI


In [ ]:
# 순수 생성 모델 테스트
pure_answer = generate_pure_answer(question)
print("Pure Generation Answer:", pure_answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Pure Generation Answer: What is the capital of the AI Kingdom?

The capital of the AI Kingdom is the AI Kingdom.

What is the AI Kingdom's currency?

The AI Kingdom's currency is the AI Kingdom's currency.

What is
